In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%pylab inline

from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from trainscript import *

%matplotlib agg
%matplotlib agg

Populating the interactive namespace from numpy and matplotlib


In [2]:
experiment_name = 'retrain_base_fromscrfitfc_labch_bengal_egyp'
experiment_shortname = 'labch_bengal_egyp'
path_base = untar_data(URLs.PETS)
path = path_base/'images'
path_experiments = Path('./experiments')
path_experiment = path_experiments/experiment_name
path_experiment.mkdir(parents = True, exist_ok=True)
csv_name = 'switch_bengal-egyptian.csv'
# csv_name = 'labels.csv'
save_newmodels = False

In [3]:
lrs_totest = [3e-5, 5e-5, 1e-4, 3e-4, 5e-4]
# lrs_totest = [1e-3]
fit_types = ['onecycle']
epochs = [10, 20]
# epochs = [1]
# epoch_long = 30
# epoch_short = 10
# epoch_long = 3
# epoch_short = 1

### Load data

In [4]:
src = ImageList.from_csv(path_base, csv_name=csv_name).split_by_rand_pct(0.4, seed=2).label_from_df(cols='label')

In [5]:
bs = 64
np.random.seed(2)

In [6]:
data = get_data(src, 224, bs)

In [7]:
gc.collect()

31

### Output Dataframe

In [8]:
pd.set_option('display.max_colwidth', None)
output_df = pd.DataFrame(columns=['basemodeltype', 'basemodelfile', 'newmodelfile', 'lr', 'fit_type', 'epoch', 'orig_metric', 'retrain_metric',
                                'orig_most_confused', 'fig_base_toplosses', 'fig_base_confmat', 'fig_train_confmat', 'fig_orig_lrfind',
                                'last_train_loss', 'last_val_loss', 'last_val_metric', 
                                'fig_train_lr', 'fig_train_losses', 'fig_train_metrics'])

### Basic sequential model

In [9]:
basemodelname='owncnn'
basemodelfile = 'basetrain-owncnn-lr3e3-ep100-fitfc05'
exp_modelpath = path_experiment/basemodelname
(exp_modelpath).mkdir(parents = True, exist_ok=True)

In [10]:
model = nn.Sequential(
    nn.Conv2d(3, 32, 3, stride = 2, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32, 64, 3, stride = 2, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 128, 3, stride = 2, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, 256, 3, stride = 2, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.Conv2d(256, 512, 3, stride = 2, padding=1),
    nn.BatchNorm2d(512),
    nn.ReLU(),
    nn.Conv2d(512, 512, 3, stride = 2, padding=1),
    nn.BatchNorm2d(512),
    nn.ReLU(),
    nn.Conv2d(512, 256, 3, stride = 2, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.Conv2d(256, 37, 3, stride = 2, padding=1),
    nn.BatchNorm2d(37),
    nn.Flatten()
)

In [11]:
learn = Learner(data, model, loss_func=nn.CrossEntropyLoss(), metrics=error_rate)

In [12]:
output_df = trainscript(learn, basemodelfile, output_df, exp_modelpath=exp_modelpath, epochs=epochs, lrs_totest=lrs_totest, fit_types=fit_types, 
                        basemodelname=basemodelname, experiment_shortname=experiment_shortname)

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.
Training: basetrain-owncnn-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr3e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.598577,2.543447,0.538904,00:13
1,0.586808,2.512915,0.542287,00:13
2,0.586167,2.494510,0.534168,00:14
3,0.533506,2.475872,0.533491,00:12
4,0.499904,2.448973,0.539919,00:10
5,0.457792,2.451541,0.535521,00:11
6,0.447861,2.432781,0.533830,00:09
7,0.433745,2.425522,0.532138,00:12
8,0.445452,2.432565,0.533153,00:15
9,0.441461,2.415468,0.533491,00:14


Training: basetrain-owncnn-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr5e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.566700,2.550396,0.539919,00:11
1,0.576448,2.512004,0.531461,00:14
2,0.516499,2.466260,0.536536,00:11
3,0.484013,2.427594,0.531123,00:11
4,0.434296,2.411035,0.532815,00:13
5,0.406486,2.392606,0.533491,00:11
6,0.372696,2.379478,0.530108,00:13
7,0.357689,2.381080,0.530785,00:13
8,0.358170,2.393049,0.530447,00:11
9,0.355193,2.369901,0.526049,00:14


Training: basetrain-owncnn-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr0.0001


...

epoch,train_loss,valid_loss,error_rate,time
0,0.578049,2.544197,0.540257,00:10
1,0.572175,2.523304,0.535859,00:11
2,0.536107,2.458751,0.537889,00:10
3,0.486692,2.433932,0.531461,00:10
4,0.427374,2.393684,0.531123,00:11
5,0.367966,2.362266,0.525710,00:11
6,0.310564,2.343064,0.524019,00:10
7,0.291948,2.327734,0.527740,00:11
8,0.264557,2.325210,0.520974,00:14
9,0.247893,2.294966,0.519959,00:11


Training: basetrain-owncnn-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0003


epoch,train_loss,valid_loss,error_rate,time
0,0.599594,2.519345,0.537212,00:11
1,0.542556,2.458704,0.537551,00:13
2,0.451365,2.394066,0.538566,00:10
3,0.361593,2.338812,0.528078,00:10
4,0.291530,2.308945,0.517930,00:10
5,0.243570,2.331077,0.527064,00:11
6,0.225175,2.263971,0.509811,00:11
7,0.192950,2.254949,0.512517,00:10
8,0.193327,2.257402,0.512855,00:10
9,0.175916,2.253136,0.502030,00:10


Training: basetrain-owncnn-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0005


epoch,train_loss,valid_loss,error_rate,time
0,0.558161,2.507154,0.533830,00:10
1,0.513869,2.439216,0.537212,00:09
2,0.402516,2.381956,0.532138,00:11
3,0.317935,2.316463,0.523681,00:10
4,0.251745,2.300370,0.518268,00:09
5,0.219614,2.269671,0.520974,00:10
6,0.187934,2.279067,0.514885,00:09
7,0.187047,2.295092,0.516915,00:10
8,0.161482,2.283984,0.512517,00:10
9,0.151788,2.278414,0.518268,00:12


### ResNet18

In [13]:
basemodelname='resnet18'
basemodelfile = 'basetrain-res18-lr3e3-ep100-fitfc05'
exp_modelpath = path_experiment/basemodelname
(exp_modelpath).mkdir(parents = True, exist_ok=True)

In [14]:
pd.set_option('display.max_colwidth', None)
output_df = pd.DataFrame(columns=['basemodeltype', 'basemodelfile', 'newmodelfile', 'lr', 'fit_type', 'epoch', 'orig_metric', 'retrain_metric',
                                'orig_most_confused', 'fig_base_toplosses', 'fig_base_confmat', 'fig_train_confmat', 'fig_orig_lrfind',
                                'last_train_loss', 'last_val_loss', 'last_val_metric', 
                                'fig_train_lr', 'fig_train_losses', 'fig_train_metrics'])

In [15]:
learn = cnn_learner(data, models.resnet18, loss_func = nn.CrossEntropyLoss(), metrics=error_rate, pretrained=False)

In [16]:
output_df = trainscript(learn, basemodelfile, output_df, exp_modelpath=exp_modelpath, epochs=epochs, lrs_totest=lrs_totest, fit_types=fit_types, 
                        basemodelname=basemodelname, experiment_shortname=experiment_shortname)

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.
Training: basetrain-res18-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr3e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.704786,1.954170,0.336604,00:14
1,0.684000,1.890294,0.339648,00:13
2,0.622243,1.835680,0.336604,00:13
3,0.560059,1.793825,0.339648,00:12
4,0.498308,1.757239,0.337280,00:12
5,0.469473,1.735783,0.334235,00:14
6,0.437477,1.713205,0.336604,00:12
7,0.430742,1.716409,0.333559,00:13
8,0.404084,1.715087,0.337618,00:15
9,0.400957,1.706484,0.332882,00:12


Training: basetrain-res18-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr5e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.710833,1.939080,0.338295,00:13
1,0.669181,1.871264,0.339986,00:14
2,0.572870,1.782743,0.337957,00:13
3,0.484891,1.723812,0.334912,00:14
4,0.435599,1.695706,0.332544,00:15
5,0.383888,1.669819,0.328823,00:12
6,0.348905,1.651412,0.328484,00:13
7,0.336420,1.637165,0.326793,00:13
8,0.314460,1.640829,0.326116,00:15
9,0.305150,1.655365,0.332206,00:13


Training: basetrain-res18-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr0.0001


...

epoch,train_loss,valid_loss,error_rate,time
0,0.702777,1.949118,0.337957,00:13
1,0.702193,1.877122,0.334574,00:12
2,0.609899,1.811817,0.339648,00:15
3,0.508094,1.730419,0.335589,00:13
4,0.407822,1.684135,0.333897,00:13
5,0.321939,1.648997,0.331867,00:16
6,0.288529,1.622878,0.324425,00:19
7,0.257970,1.605747,0.320704,00:16
8,0.233353,1.584016,0.316306,00:15
9,0.213959,1.566611,0.317659,00:13


Training: basetrain-res18-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0003


epoch,train_loss,valid_loss,error_rate,time
0,0.662225,1.893677,0.339310,00:13
1,0.579175,1.765746,0.336604,00:13
2,0.459070,1.676258,0.332544,00:13
3,0.331459,1.630272,0.327131,00:13
4,0.256514,1.591178,0.322395,00:13
5,0.199366,1.557930,0.315291,00:13
6,0.163430,1.523628,0.313261,00:12
7,0.149683,1.527467,0.317321,00:12
8,0.134199,1.549104,0.319012,00:12
9,0.116926,1.514510,0.308187,00:13


Training: basetrain-res18-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0005


epoch,train_loss,valid_loss,error_rate,time
0,0.657982,1.878299,0.335927,00:12
1,0.518518,1.715270,0.337280,00:13
2,0.379408,1.646960,0.325440,00:13
3,0.257861,1.578681,0.322057,00:12
4,0.196954,1.635110,0.322733,00:13
5,0.166764,1.565946,0.312923,00:13
6,0.139089,1.625349,0.324763,00:13
7,0.107796,1.558766,0.313599,00:13
8,0.098204,1.580248,0.317659,00:13
9,0.094660,1.580542,0.317659,00:13


In [17]:
learn.destroy()
gc.collect()

this Learner object self-destroyed - it still exists, but no longer usable


0

### ResNet34

In [18]:
basemodelname='resnet34'
basemodelfile = 'basetrain-res34-lr3e3-ep100-fitfc05'
exp_modelpath = path_experiment/basemodelname
(exp_modelpath).mkdir(parents = True, exist_ok=True)

In [19]:
pd.set_option('display.max_colwidth', None)
output_df = pd.DataFrame(columns=['basemodeltype', 'basemodelfile', 'newmodelfile', 'lr', 'fit_type', 'epoch', 'orig_metric', 'retrain_metric',
                                'orig_most_confused', 'fig_base_toplosses', 'fig_base_confmat', 'fig_train_confmat', 'fig_orig_lrfind',
                                'last_train_loss', 'last_val_loss', 'last_val_metric', 
                                'fig_train_lr', 'fig_train_losses', 'fig_train_metrics'])

In [20]:
learn = cnn_learner(data, models.resnet34, loss_func = nn.CrossEntropyLoss(), metrics=error_rate, pretrained=False)

In [21]:
output_df = trainscript(learn, basemodelfile, output_df, exp_modelpath=exp_modelpath, epochs=epochs, lrs_totest=lrs_totest, fit_types=fit_types, 
                        basemodelname=basemodelname, experiment_shortname=experiment_shortname)

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.
Training: basetrain-res34-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr3e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.676717,8.835951,0.329161,00:18
1,0.659817,4.726432,0.326793,00:18
2,0.603475,4.885137,0.330853,00:18
3,0.509503,5.667296,0.326455,00:18
4,0.479144,4.985268,0.322733,00:20
5,0.423543,4.605580,0.327470,00:23
6,0.388791,6.578213,0.324087,00:18
7,0.386680,7.133181,0.324763,00:18
8,0.364705,6.885777,0.324425,00:25
9,0.355515,6.485638,0.326793,00:18


Training: basetrain-res34-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr5e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.672022,8.927610,0.329838,00:21
1,0.614725,6.102273,0.326116,00:18
2,0.545591,7.535857,0.329499,00:19
3,0.448535,11.046088,0.325101,00:23
4,0.380888,4.919454,0.325440,00:19
5,0.338211,7.215968,0.322057,00:18
6,0.303153,7.836712,0.324425,00:22
7,0.285249,7.314594,0.321719,00:18
8,0.308139,11.048719,0.323410,00:19
9,0.281466,8.413129,0.324087,00:21


Training: basetrain-res34-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr0.0001


...

epoch,train_loss,valid_loss,error_rate,time
0,0.659566,3.969178,0.326116,00:21
1,0.630340,3.512683,0.327808,00:19
2,0.580793,4.808032,0.328146,00:18
3,0.487186,5.175996,0.325101,00:26
4,0.384924,4.344811,0.319012,00:18
5,0.298846,6.136150,0.320365,00:19
6,0.244883,5.212355,0.317659,00:20
7,0.211739,9.324450,0.315629,00:20
8,0.203524,5.983174,0.315291,00:18
9,0.169154,7.388222,0.310555,00:21


Training: basetrain-res34-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0003


epoch,train_loss,valid_loss,error_rate,time
0,0.658678,5.463017,0.328146,00:20
1,0.579476,5.055456,0.324425,00:18
2,0.412873,7.733217,0.329161,00:20
3,0.303015,7.265040,0.315291,00:21
4,0.223259,5.182297,0.307848,00:18
5,0.183642,6.571262,0.302774,00:19
6,0.154878,4.053955,0.298376,00:18
7,0.130882,7.496126,0.301759,00:18
8,0.111688,6.527731,0.291272,00:19
9,0.106782,4.182958,0.294655,00:18


Training: basetrain-res34-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0005


epoch,train_loss,valid_loss,error_rate,time
0,0.631157,4.161143,0.323748,00:19
1,0.503904,4.618011,0.323072,00:19
2,0.343996,8.958604,0.321380,00:19
3,0.234639,9.603844,0.320704,00:19
4,0.173184,5.759138,0.302774,00:18
5,0.143088,4.408000,0.304127,00:19
6,0.110091,4.739367,0.304127,00:20
7,0.108123,7.076875,0.309540,00:19
8,0.089094,12.759892,0.316982,00:21
9,0.089336,7.441543,0.302436,00:18


In [22]:
learn.destroy()
gc.collect()

this Learner object self-destroyed - it still exists, but no longer usable


0

### ResNet50

In [23]:
basemodelname='resnet50'
basemodelfile = 'basetrain-res50-lr3e3-ep100-fitfc05'
exp_modelpath = path_experiment/basemodelname
(exp_modelpath).mkdir(parents = True, exist_ok=True)

In [24]:
pd.set_option('display.max_colwidth', None)
output_df = pd.DataFrame(columns=['basemodeltype', 'basemodelfile', 'newmodelfile', 'lr', 'fit_type', 'epoch', 'orig_metric', 'retrain_metric',
                                'orig_most_confused', 'fig_base_toplosses', 'fig_base_confmat', 'fig_train_confmat', 'fig_orig_lrfind',
                                'last_train_loss', 'last_val_loss', 'last_val_metric', 
                                'fig_train_lr', 'fig_train_losses', 'fig_train_metrics'])

In [25]:
learn = cnn_learner(data, models.resnet50, loss_func = nn.CrossEntropyLoss(), metrics=error_rate, pretrained=False)

In [26]:
output_df = trainscript(learn, basemodelfile, output_df, exp_modelpath=exp_modelpath, epochs=epochs, lrs_totest=lrs_totest, fit_types=fit_types, 
                        basemodelname=basemodelname, experiment_shortname=experiment_shortname)

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.
Training: basetrain-res50-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr3e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.620608,1.776924,0.384303,00:34
1,0.633348,1.753211,0.377537,00:33
2,0.589854,1.704780,0.381597,00:33
3,0.547723,1.677515,0.380244,00:33
4,0.523666,1.642088,0.375846,00:33
5,0.508757,1.634961,0.377199,00:34
6,0.491906,1.622953,0.376184,00:35
7,0.463797,1.615988,0.378214,00:33
8,0.453466,1.613123,0.374831,00:33
9,0.473382,1.621567,0.375846,00:33


Training: basetrain-res50-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr5e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.638179,1.771068,0.379905,00:33
1,0.610529,1.714855,0.381258,00:35
2,0.584156,1.673593,0.376861,00:33
3,0.544328,1.625131,0.379229,00:34
4,0.491821,1.595182,0.381258,00:34
5,0.455245,1.590400,0.377876,00:33
6,0.437645,1.570485,0.375169,00:33
7,0.425634,1.570525,0.374154,00:33
8,0.415718,1.564365,0.376184,00:33
9,0.411878,1.553161,0.375507,00:33


Training: basetrain-res50-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr0.0001


...

epoch,train_loss,valid_loss,error_rate,time
0,0.639366,1.771493,0.379567,00:36
1,0.600337,1.738312,0.381935,00:33
2,0.568628,1.686071,0.379567,00:34
3,0.548575,1.642612,0.383965,00:34
4,0.485131,1.581654,0.377876,00:33
5,0.438759,1.561445,0.376184,00:33
6,0.391501,1.525033,0.370771,00:33
7,0.362093,1.496659,0.362991,00:33
8,0.329665,1.493573,0.362991,00:33
9,0.321973,1.474323,0.358254,00:33


Training: basetrain-res50-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0003


epoch,train_loss,valid_loss,error_rate,time
0,0.623588,1.750544,0.382612,00:33
1,0.583057,1.681415,0.376184,00:33
2,0.488899,1.599085,0.380920,00:34
3,0.421189,1.545201,0.377537,00:34
4,0.376386,1.497941,0.366035,00:33
5,0.334615,1.468429,0.359269,00:34
6,0.294672,1.472954,0.352165,00:33
7,0.279324,1.473751,0.351827,00:33
8,0.270930,1.468350,0.350474,00:33
9,0.230321,1.457716,0.351827,00:33


Training: basetrain-res50-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0005


epoch,train_loss,valid_loss,error_rate,time
0,0.631027,1.722213,0.379905,00:35
1,0.560390,1.635927,0.380920,00:33
2,0.453130,1.569785,0.375169,00:35
3,0.374876,1.519574,0.360961,00:33
4,0.338261,1.501407,0.361976,00:33
5,0.320991,1.525673,0.365020,00:33
6,0.284973,1.495961,0.358931,00:33
7,0.257931,1.454143,0.351489,00:33
8,0.245202,1.494781,0.360622,00:34
9,0.232958,1.475565,0.349120,00:33


In [27]:
learn.destroy()
gc.collect()

this Learner object self-destroyed - it still exists, but no longer usable


0

### ResNet101

In [28]:
basemodelname='resnet101'
basemodelfile = 'basetrain-res101-lr3e3-ep100-fitfc05'
exp_modelpath = path_experiment/basemodelname
(exp_modelpath).mkdir(parents = True, exist_ok=True)

In [29]:
pd.set_option('display.max_colwidth', None)
output_df = pd.DataFrame(columns=['basemodeltype', 'basemodelfile', 'newmodelfile', 'lr', 'fit_type', 'epoch', 'orig_metric', 'retrain_metric',
                                'orig_most_confused', 'fig_base_toplosses', 'fig_base_confmat', 'fig_train_confmat', 'fig_orig_lrfind',
                                'last_train_loss', 'last_val_loss', 'last_val_metric', 
                                'fig_train_lr', 'fig_train_losses', 'fig_train_metrics'])

In [30]:
learn = cnn_learner(data, models.resnet101, loss_func = nn.CrossEntropyLoss(), metrics=error_rate, pretrained=False)

In [31]:
output_df = trainscript(learn, basemodelfile, output_df, exp_modelpath=exp_modelpath, epochs=epochs, lrs_totest=lrs_totest, fit_types=fit_types, 
                        basemodelname=basemodelname, experiment_shortname=experiment_shortname)

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.
Training: basetrain-res101-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr3e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.600669,74.922707,0.404601,00:57
1,0.615489,64.559273,0.404939,00:51
2,0.610913,120.791115,0.404601,00:52
3,0.573641,151.757690,0.403586,00:51
4,0.587752,66.300148,0.404263,00:53
5,0.560184,117.004829,0.407984,00:53
6,0.582208,120.161011,0.405277,00:52
7,0.557062,110.846794,0.408322,00:51
8,0.538629,162.403610,0.405954,00:53
9,0.536678,90.986931,0.400541,00:52


Training: basetrain-res101-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr5e-05


epoch,train_loss,valid_loss,error_rate,time
0,0.623846,63.867710,0.404263,00:53
1,0.603941,86.286034,0.402571,00:53
2,0.594962,143.987473,0.403586,00:52
3,0.559150,140.228775,0.407984,00:51
4,0.543035,140.160645,0.407307,00:51
5,0.537299,141.943954,0.400880,00:51
6,0.496404,94.221985,0.398511,00:53
7,0.496360,266.092896,0.401556,00:52
8,0.475605,256.950256,0.400203,00:51
9,0.488047,104.388466,0.400541,00:51


Training: basetrain-res101-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep10-onecycle-lr0.0001


...

epoch,train_loss,valid_loss,error_rate,time
0,0.626300,97.473541,0.404939,00:51
1,0.635408,123.102409,0.406292,00:53
2,0.606391,86.097015,0.404601,00:52
3,0.580272,209.865692,0.404601,00:55
4,0.554101,194.972900,0.402909,00:53
5,0.510917,195.187515,0.396143,00:53
6,0.462899,143.016754,0.394114,00:52
7,0.448070,263.099091,0.392084,00:52
8,0.435095,129.068863,0.390054,00:58
9,0.420976,345.960876,0.393437,00:55


Training: basetrain-res101-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0003


epoch,train_loss,valid_loss,error_rate,time
0,0.640831,151.906738,0.407645,01:00
1,0.586816,123.412628,0.403248,00:52
2,0.561886,264.879486,0.408322,00:52
3,0.478795,137.316025,0.400880,00:52
4,0.431920,520.023193,0.395805,00:52
5,0.408009,395.490753,0.393437,00:51
6,0.375150,498.389374,0.390054,00:54
7,0.361019,1173.775879,0.385318,00:52
8,0.341905,981.757019,0.380582,00:51
9,0.304157,1292.150635,0.381935,00:51


Training: basetrain-res101-lr3e3-ep100-fitfc05_labch_bengal_egyp-ep20-onecycle-lr0.0005


epoch,train_loss,valid_loss,error_rate,time
0,0.645264,89.934975,0.408999,00:50
1,0.576960,131.111099,0.406292,00:52
2,0.518607,176.662338,0.400203,00:51
3,0.456454,271.183136,0.404939,00:53
4,0.429600,272.842468,0.393099,00:56
5,0.386096,160.516144,0.388024,00:52
6,0.372658,229.684525,0.383288,00:51
7,0.361775,231.809204,0.381935,00:53
8,0.307147,91.626915,0.379229,00:51
9,0.285598,129.995132,0.379229,00:51


In [32]:
learn.destroy()
gc.collect()

this Learner object self-destroyed - it still exists, but no longer usable


0